In [ ]:
# @title بناء الجدول التصنيفي للوظائف التداولية (Typology Table Construction)
import json
import pandas as pd
from google.colab import drive

# ---------------------------------------------------------
# 1. SETUP & LOAD DATA
# ---------------------------------------------------------
print("Mounting Google Drive...")
drive.mount('/content/drive')

path_similes = '/content/drive/MyDrive/ Pragmatic/corpus/similes_data.json'
path_metaphors = '/content/drive/MyDrive/Pragmatic/corpus/metaphors_data.json'

def load_data(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except:
        return []

data_similes = load_data(path_similes)
data_metaphors = load_data(path_metaphors)
all_data = data_similes + data_metaphors

print(f"✅ Loaded {len(all_data)} records for qualitative extraction.")

# ---------------------------------------------------------
# 2. DEFINING THE TYPOLOGY LOGIC
# ---------------------------------------------------------
# نحن نبحث عن أمثلة محددة تخدم السردية المطلوبة في 4.2
# سنقوم بالبحث عن سجلات تحتوي على الوسوم المطلوبة + كلمات مفتاحية في الشرح

target_categories = [
    {
        "Main_Function": "Directive Functions (التوجيه)",
        "Strategy": "Force Boosting (التشديد)",
        "Tag": "Warning & Intimidation",
        "Definition": "Utilizing figurative intensity to amplify the illocutionary force of threats and warnings, moving from simple prohibition to cognitive deterrence.",
        "Keywords": ["تهويل", "زجر", "ردع", "تخويف"]
    },
    {
        "Main_Function": "Directive Functions (التوجيه)",
        "Strategy": "Mitigation & Softening (التلطف)",
        "Tag": "Incentive & Attraction",
        "Definition": "Employing pleasant imagery to soften directive acts, transforming bare commands into inviting aesthetic experiences.",
        "Keywords": ["تشويق", "ترغيب", "تحبيب", "تلطف"]
    },
    {
        "Main_Function": "Persuasive & Argumentative",
        "Strategy": "Cognitive Bridging (التقريب)",
        "Tag": "Clarification & Imagery",
        "Definition": "Using metaphor as an epistemic tool to bridge the gap between abstract theological concepts and human sensory experience.",
        "Keywords": ["تقريب", "تجسيد", "تمثيل", "عقلي"]
    },
    {
        "Main_Function": "Persuasive & Argumentative",
        "Strategy": "Evidential Argumentation (الاستدلال)",
        "Tag": "Argumentation & Persuasion",
        "Definition": "Deploying imagery as logical evidence (burhān) to bind the addressee to a conclusion through analogical reasoning.",
        "Keywords": ["حجة", "دليل", "برهان", "إلزام"]
    },
    {
        "Main_Function": "Affective & Evaluative",
        "Strategy": "Negative Stance (Delegitimization)",
        "Tag": "Condemnation & Criticism",
        "Definition": "Constructing an exclusionary stance towards antagonists through imagery that evokes disgust, contempt, or absurdity.",
        "Keywords": ["سخرية", "تهكم", "تحقير", "تقبيح"]
    },
    {
        "Main_Function": "Affective & Evaluative",
        "Strategy": "Positive Stance (Legitimization)",
        "Tag": "Glorification & Exaltation",
        "Definition": "Establishing a reverent stance towards the Divine or believers through imagery of height, light, and stability.",
        "Keywords": ["تعظيم", "تمجيد", "تنزيه", "مدح"]
    }
]

# ---------------------------------------------------------
# 3. EXTRACTION ENGINE
# ---------------------------------------------------------
typology_rows = []

for category in target_categories:
    best_example = None
    max_score = 0

    for record in all_data:
        # البحث داخل التحليل البلاغي
        similes = record.get('rhetorical_analysis', {}).get('similes', [])
        for item in similes:
            functions = item.get('functions', [])

            # التحقق من وجود الوظيفة المطلوبة
            matched_func = None
            for func in functions:
                if func.get('pragmatic_function_tage') == category['Tag']:
                    matched_func = func
                    break

            if matched_func:
                # حساب "جودة" المثال:
                # نفضل المثال الذي يحتوي شرحه (detail) على الكلمات المفتاحية الدقيقة
                text_content = matched_func.get('detail', '')
                score = 0
                for kw in category['Keywords']:
                    if kw in text_content:
                        score += 1

                # نفضل الآيات القصيرة المناسبة للجدول
                ayah_len = len(record['metadata']['ayah_text_uthmani'])
                if ayah_len < 150: score += 1

                # تحديث أفضل مثال
                if score > max_score:
                    max_score = score
                    # استخراج الشرح المقتضب
                    analysis_snippet = text_content[:150] + "..." if len(text_content) > 150 else text_content

                    best_example = {
                        "Main Function": category['Main_Function'],
                        "Pragmatic Strategy": category['Strategy'],
                        "Pragmatic Definition": category['Definition'],
                        "Representative Example": f"\"{record['metadata']['ayah_text_uthmani']}\" ({record['metadata']['chapter_no']}:{record['metadata']['verse_no']})",
                        "Analysis Snippet": analysis_snippet
                    }

    if best_example:
        typology_rows.append(best_example)

# ---------------------------------------------------------
# 4. GENERATE TABLE (VISUALIZATION)
# ---------------------------------------------------------
df_typology = pd.DataFrame(typology_rows)

print("\n" + "="*80)
print("📋 TABLE 1: FUNCTIONAL TYPOLOGY OF QUR'ANIC FIGURATIVE LANGUAGE")
print("="*80)

# عرض الجدول بشكل منسق (للمراجعة)
pd.set_option('display.max_colwidth', None)
display(df_typology[['Main Function', 'Pragmatic Strategy', 'Pragmatic Definition', 'Representative Example']])

# تصدير إلى نص منسق (لنسخه إلى الورقة)
print("\n📝 [Text Format for Paper Insertion]:\n")
for index, row in df_typology.iterrows():
    print(f"🔹 {row['Main Function']} -> {row['Pragmatic Strategy']}")
    print(f"   • Definition: {row['Pragmatic Definition']}")
    print(f"   • Example: {row['Representative Example']}")
    print(f"   • Analysis: {row['Analysis Snippet']}")
    print("-" * 50)

Mounting Google Drive...
Mounted at /content/drive
✅ Loaded 965 records for qualitative extraction.

📋 TABLE 1: FUNCTIONAL TYPOLOGY OF QUR'ANIC FIGURATIVE LANGUAGE


,Main Function,Pragmatic Strategy,Pragmatic Definition,Representative Example
0,Directive Functions (التوجيه),Force Boosting (التشديد),"Utilizing figurative intensity to amplify the illocutionary force of threats and warnings, moving from simple prohibition to cognitive deterrence.","""خَتَمَ ٱللَّهُ عَلَىٰ قُلُوبِهِمْ وَعَلَىٰ سَمْعِهِمْ ۖ وَعَلَىٰٓ أَبْصَٰرِهِمْ غِشَٰوَةٌ ۖ وَلَهُمْ عَذَابٌ عَظِيمٌ"" (2:7)"
1,Directive Functions (التوجيه),Mitigation & Softening (التلطف),"Employing pleasant imagery to soften directive acts, transforming bare commands into inviting aesthetic experiences.","""وَوَصَّىٰ بِهَآ إِبْرَٰهِۦمُ بَنِيهِ وَيَعْقُوبُ يَٰبَنِىَّ إِنَّ ٱللَّهَ ٱصْطَفَىٰ لَكُمُ ٱلدِّينَ فَلَا تَمُوتُنَّ إِلَّا وَأَنتُم مُّسْلِمُونَ"" (2:132)"
2,Persuasive & Argumentative,Cognitive Bridging (التقريب),Using metaphor as an epistemic tool to bridge the gap between abstract theological concepts and human sensory experience.,"""ٱهْدِنَا ٱلصِّرَٰطَ ٱلْمُسْتَقِيمَ"" (1:6)"
3,Persuasive & Argumentative,Evidential Argumentation (الاستدلال),Deploying imagery as logical evidence (burhān) to bind the addressee to a conclusion through analogical reasoning.,"""وَقَالُوا۟ قُلُوبُنَا غُلْفٌۢ ۚ بَل لَّعَنَهُمُ ٱللَّهُ بِكُفْرِهِمْ فَقَلِيلًا مَّا يُؤْمِنُونَ"" (2:88)"
4,Affective & Evaluative,Negative Stance (Delegitimization),"Constructing an exclusionary stance towards antagonists through imagery that evokes disgust, contempt, or absurdity.","""خَتَمَ ٱللَّهُ عَلَىٰ قُلُوبِهِمْ وَعَلَىٰ سَمْعِهِمْ ۖ وَعَلَىٰٓ أَبْصَٰرِهِمْ غِشَٰوَةٌ ۖ وَلَهُمْ عَذَابٌ عَظِيمٌ"" (2:7)"
5,Affective & Evaluative,Positive Stance (Legitimization),"Establishing a reverent stance towards the Divine or believers through imagery of height, light, and stability.","""ٱهْدِنَا ٱلصِّرَٰطَ ٱلْمُسْتَقِيمَ"" (1:6)"



📝 [Text Format for Paper Insertion]:

🔹 Directive Functions (التوجيه) -> Force Boosting (التشديد)
   • Definition: Utilizing figurative intensity to amplify the illocutionary force of threats and warnings, moving from simple prohibition to cognitive deterrence.
   • Example: "خَتَمَ ٱللَّهُ عَلَىٰ قُلُوبِهِمْ وَعَلَىٰ سَمْعِهِمْ ۖ وَعَلَىٰٓ أَبْصَٰرِهِمْ غِشَٰوَةٌ ۖ وَلَهُمْ عَذَابٌ عَظِيمٌ" (2:7)
   • Analysis: تَعْمَلُ الِاسْتِعَارَةُ عَلَى وَصْفِ حَالَةِ الْكَافِرِينَ بِصُورَةٍ مُرَوِّعَةٍ تُثِيرُ النُّفُورَ الشَّدِيدَ مِنْهَا، إِذْ لَا شَيْءَ أَشَدُّ وَطْأ...
--------------------------------------------------
🔹 Directive Functions (التوجيه) -> Mitigation & Softening (التلطف)
   • Definition: Employing pleasant imagery to soften directive acts, transforming bare commands into inviting aesthetic experiences.
   • Example: "وَوَصَّىٰ بِهَآ إِبْرَٰهِۦمُ بَنِيهِ وَيَعْقُوبُ يَٰبَنِىَّ إِنَّ ٱللَّهَ ٱصْطَفَىٰ لَكُمُ ٱلدِّينَ فَلَا تَمُوتُنَّ إِلَّا وَأَنتُم مُّسْلِمُونَ" (2:132)
   • A